Notebook for the one-hot autoencoder

In [1]:
from communications import *
from model import *
import numpy as np
import matplotlib.pyplot as plt
import os.path

%load_ext autoreload
%autoreload 2

In [2]:
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
X_train,y_train=np.eye(256),np.eye(256)
#X_train,y_train=createBitVectors(),createBitVectors()
#X_train,y_train=createBitVectors(),2.0*createPolarCodewords()-1


In [4]:
SNR=3
add_name='_ohtest'

In [ ]:
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder(5*256,5*256,AWGN,SNR,use_BN=False,use_LN=False)

In [ ]:
boundaries = [2**16,2**17]
values = [1e-4,1e-5,1e-6]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder2(5*256,5*256,AWGN,SNR,use_BN=True,use_LN=False,lr=1e-4,encoder_activation='tanh',hidden_activation='relu',optim=Adam,lookahead=False,gradient_centralization=False)

In [ ]:
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder(5*256,5*256,multi_AWGN,use_BN=False,use_LN=False)

In [5]:
def schedule(epoch,lr):
    if epoch<2**13:
        return 1e-3
    if epoch<3*2**16:
        return 1e-5
    return 1e-6
    

scheduler=tf.keras.callbacks.LearningRateScheduler(schedule)


In [ ]:
from tensorflow.keras.models import load_model
#autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('normal','normal',[64,128],[128,64,32],AWGN,SNR,use_BN=True,use_LN=False,lr=1e-3,encoder_activation='tanh',hidden_activation=mish,optim=RectifiedAdam,lookahead=True,gradient_centralization=True)
encoderAWGN=load_model('AWGNEncoder_3_relu.h5',custom_objects={'AWGN':AWGN,'normLayer':normLayer})
#autoencoderAWGN,encoderAWGN,decoderAWGN=recompileAutoencoder('normal','normal',encoderAWGN,decoderAWGN,AWGN,noise_param=SNR,optim=Adam,lookahead=True,gradient_centralization=True,lr=1e-5)

In [6]:
'''Possible additional layers:
-normLayer:mean=0 and std=1
-pseudoNorm:mean=0 and max value=1
-forceNorm: outputs 1 or -1 not differentiable
-normLoss: adds mean and values deviations to loss'''

autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('onehot','onehot',[1280],[64,1280],AWGN,SNR,use_BN=True,use_LN=False,use_WN=False,
                                                    lr=1e-4,encoder_activation='tanh',hidden_activation=mish,
                                                    optim=RectifiedAdam,lookahead=True,gradient_centralization=True,
                                                    compile_autoencoder=True,compile_encoder=False,compile_decoder=False,
                                                    additional_layers=[normLoss])

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint('AWGNAutoencoderCheckpoint'+add_name+'.h5',monitor='loss',save_best_only=True)
history=encoderAWGN.fit(x=X_train,y=y_train,batch_size=256,epochs=2**15,callbacks=[checkpoint,scheduler],verbose=2)
save_obj(history.history,'history'+add_name)
encoderAWGN.save('AWGNEncoder'+add_name+'.h5')

In [ ]:
print(y_train)

In [ ]:
from tensorflow.keras.models import load_model
#autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('normal','normal',[64,128],[128,64,32],AWGN,SNR,use_BN=True,use_LN=False,lr=1e-3,encoder_activation='tanh',hidden_activation=mish,optim=RectifiedAdam,lookahead=True,gradient_centralization=True)
encoderAWGN=load_model('AWGNEncoder_3_polarencoder.h5',custom_objects={'AWGN':AWGN,'polar_metric':polar_metric})
#autoencoderAWGN,encoderAWGN,decoderAWGN=recompileAutoencoder('normal','normal',encoderAWGN,decoderAWGN,AWGN,noise_param=SNR,optim=Adam,lookahead=True,gradient_centralization=True,lr=1e-5)

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint('AWGNAutoencoderCheckpoint'+add_name+'.h5',monitor='loss',save_best_only=True)
history=autoencoderAWGN.fit(x=X_train,y=y_train,batch_size=256,epochs=2**18,callbacks=[checkpoint,scheduler],verbose=0)
save_obj(history.history,'history'+add_name)
autoencoderAWGN.save('AWGNAutoencoder'+add_name+'.h5')
encoderAWGN.save('AWGNEncoder'+add_name+'.h5')
decoderAWGN.save('AWGNDecoder'+add_name+'.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [14]:
print(encoderAWGN.predict(X_train))

[[ 0.9999912  -0.9999937  -0.999998   ...  0.9999657  -0.9999949
  -0.99999577]
 [ 0.9999928   0.999992   -0.999979   ...  0.99999326  0.9999953
  -0.99999   ]
 [ 0.9999888   0.99998707 -0.99999744 ... -0.99999726 -0.99999785
   1.        ]
 ...
 [ 0.99622923 -0.9996762   0.9999909  ...  1.         -0.9999699
  -0.99972016]
 [ 0.9999928  -0.9999916  -0.9999918  ... -0.99999374 -0.99999607
  -0.9999962 ]
 [-0.9999264   0.99999285  0.9999908  ...  0.99999726 -0.99926764
  -0.999947  ]]


In [ ]:
computeVarianceAWGN(encoderAWGN,do_polar=True,print_code=True,round=False)

In [ ]:
testLinearAWGN(encoderAWGN)

In [ ]:
np.count_non_zero(np.round(encoderAWGN.predict(np.eye(256))))

In [ ]:
print()